In [1]:
from nlgeval import compute_individual_metrics, compute_metrics
from bleu_eval_new import count_common, count_hit, count_score, get_score
import pickle
import numpy as np
import nltk


In [7]:

results = open('./result/best_save_bert.out.txt', 'r', encoding='utf-8').readlines()
print(len(results))
print(results[1016])

2031
  patient was successfully extubated , and chest tube removed without complication following resolution of air leak .



In [8]:
from tqdm import tqdm
import nltk
import pickle
import re

def obtain(txt, exist_dic, src, tar):
    for para in tqdm(txt):
        sentences = para.split('\n')
        tars = []
        dics = []
        if len(tar)==490:
            print('here')
        if len(sentences) < 2 or len(sentences[0]) < 3 or len(sentences[1]) < 3:
            continue
        for sid, sentence in enumerate(sentences):
            if sid == 0:
                src.append(sentence+' EOST')
            else:
                cudic = {}
                sentence = sentence[2:]
                text = re.sub('\[[^\[\]]*\]', '', sentence)
                pairs = re.findall('[^\[\] ]+\[[^\[\]]+\]', sentence)
                for pair in pairs:
                    pair = re.split('[\[\]]', pair)
                    cudic[pair[0]] = pair[1]
                words = nltk.word_tokenize(text)
                for wid, word in enumerate(words):
                    if word in cudic.keys():
                        words[wid] = cudic[word]
                new_text = ''
                for word in words:
                    new_text += word
                    new_text += ' '
                tars.append(new_text)
                dics.append(cudic)

        tar.append(tars)
        exist_dic.append(dics)

txt = open('./data_tmp/test(2030).txt', 'r').read()
txt = txt.lower()
txt = txt.split('\n\n')
txtv = txt[0:len(txt)//2]
txtt = txt[len(txt)//2:]
src = []
tar = []
exist_dic = []
obtain(txtv, exist_dic, src, tar)
pickle.dump(exist_dic, open('./testdata/val_dic.pkl', 'wb'))
pickle.dump(src, open('./testdata/val.pro.pkl', 'wb'))
pickle.dump(tar, open('./testdata/val.cus.pkl', 'wb'))

src = []
tar = []
exist_dic = []
obtain(txtt, exist_dic, src, tar)
pickle.dump(exist_dic, open('./testdata/test_dic.pkl', 'wb'))
pickle.dump(src, open('./testdata/test.pro.pkl', 'wb'))
pickle.dump(tar, open('./testdata/test.cus.pkl', 'wb'))

 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                         | 757/1015 [00:00<00:00, 1825.24it/s]

here


 72%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                              | 729/1016 [00:00<00:00, 1795.04it/s]

here


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 1765.85it/s]


In [9]:
# results = open('./epoch30/result/best_save_bert.out.txt', 'r', encoding='utf-8').readlines()
# results = results[1015:]
# results = open('result/tar_sts.txt', 'r', encoding='utf-8').readlines()
# results = open('./epoch30-split/result/best_save_bert.out.txt', 'r', encoding='utf-8').readlines()
results = results[1015:]
print(len(results))

sources = open('testdata/test.moses.pro', 'r').readlines()
sources = [x.replace('\n', '') for x in sources]
ref = pickle.load(open('testdata/test.cus.pkl', 'rb'))
dics = pickle.load(open('testdata/test_dic.pkl', 'rb'))
sen2code = pickle.load(open('data/sen2code.pkl', 'rb'))
sen2code_new = {}
for key, value in sen2code.items():
    sen2code_new[key.lower()] = value
del sen2code
count = 0
code_exist = {}
for source in sources:
    try:
        if sen2code_new[source] not in code_exist.keys():
            code_exist[sen2code_new[source]] = 1
        else:
            code_exist[sen2code_new[source]] += 1
    except:
        count+=1
print(count)
print(len(code_exist.keys()))
test_subjects = np.array(results)
test_targets = np.array(ref)
test_dics = np.array(dics)
len_sen = [len(nltk.word_tokenize(x)) for x in sources]
len_sen = np.array(len_sen)
print(len_sen.mean(),len_sen.max() ,len_sen.min())
len_spilt = [(0, 100000)]


1016
58
394
19.93996062992126 67 7


e:\ljx\step3-v4\venv\lib\site-packages\ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
e:\ljx\step3-v4\venv\lib\site-packages\ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
test_txt = open('./data_tmp/test(2030).txt', 'r', encoding = 'utf-8').read().split('\n\n')
test_srcs = []
for u in test_txt:
    sts = u.split('\n')
    test_srcs.append(sts[0])
test_srcs = test_srcs[1015:]

test_tars = pickle.load(open('./data_tmp/test_tars.pkl', 'rb'))
test_tars = test_tars[1015:]

for (a,b,c) in zip(test_srcs, test_tars, results):
    print('src: ',a)
    print('tar: ', b[0])
    print('out: ', c.strip())
    print()
    
    

src:  She was taken to IR where 3 peripheral branches of the right renal artery was embolized , and a HD line was placed under sterile procedure in the RIJ .
tar:  She was taken to Interventional radiology where 3 peripheral branches of the right kidney artery was embolized , and a Hemodialysis line was placed under sterile procedure in the Right Internal Jugular .
out:  she was taken to  outside hospital where 3 peripheral branches of the right kidney artery was embolized , and a  flexible catheter line was placed under sterile procedure in the  emergency department .

src:  Pt was successfully extubated , and chest tube removed without complication following resolution of air leak .
tar:  patient was successfully removed from ventilator , and chest tube removed without complication following resolution of air leak .
out:  patient was successfully extubated , and chest tube removed without complication following resolution of air leak .

src:  Encourage/Assist with PO intake and give 

tar:  Patient 732 is a 59 year old female with metastatic breast cancer with recent hospitalization for pulmonary embolism who presents with shortness of breath and acute agitation .
out:  known lastname 732 is a 59 year old female with metastatic breast cancer with recent hospitalization for  lung disease who presents with  low blood pressure and acute agitation .

src:  On the floor , he was re-started on his coumadin with a target INR of 2.0 - 3.0 .
tar:  On the floor , he was re-started on his coumadin with a target international normalized ratio of 2.0 - 3.0 .
out:  on the floor , he was re - started on his coumadin with a target international normalized ratio of 2 . 0 - 3 . 0 .

src:  42 year old female with h/o SDH 2-26 , EtOH abuse , ?
tar:  42 year old female with history of  subdural hematoma 2-26 , ethanol abuse , ?
out:  42 year old female with history of 2 - 26 , abuse , ?

src:  REVIEW OF SYSTEMS + per HPI , some calf cramping and dysphagia On review of systems , she deni

out:  a pulmunary embolism was noted on the admission  body imaging scan .

src:  At that time the lesion expressed a large amount of largely serous fluid .
tar:  At that time the lesion expressed a large amount of largely serous fluid .
out:  at that time the lesion expressed a large amount of largely serous fluid .

src:  Her mental status and oxygen improved as her bicarb normalized .
tar:  Her mental status and oxygen improved as her sodium bicarbonate normalized .
out:  her mental status and oxygen improved as her bicarbonate normalized .

src:  On ROS , he denies pain other than his baseline MSK complaints .
tar:  On review of systems , he denies pain other than his baseline musculoskeletal complaints .
out:  on review of systems , he denies pain other than his baseline medullary sponge kidney uk : musculoskeletal complaints .

src:  51 F with history significant for COPD with multiple admissions to Hospital3 2737 who was again admitted there on 2160-9-10 for COPD exacerbation .


tar:  The patient 's chief complaint was right flank pain .
out:  the patient ' s chief complaint was right flank pain .

src:  At the time of discharge the patient was able to ambulate while on room air without desaturation .
tar:  At the time of discharge the patient was able to ambulate while on room air without desaturation .
out:  at the time of discharge the patient was able to ambulate while on room air without desaturation .

src:  she was treated empirically with vanc/zosyn which was narrowed to cefepime ; this was discontinued prior to discharge .
tar:  she was treated empirically with vanc/zosyn which was narrowed to cefepime ; this was discontinued prior to discharge .
out:  she was treated empirically with vanc / zosyn which was narrowed to cefepime ; this was discontinued prior to discharge .

src:  She has not been on any prolonged trips , had any recent surgeries , been immobilized recently , and has never felt a pain similar to this one .
tar:  She has not been on any 

tar:  Unclear baseline mental status , per Emergency Medical Service(s) patient visiting from State 108 .
out:  unclear baseline mental status , per emergency medical service ( s ) patient visiting from state 108 .

src:  Head CT showed a small contusion with a small subarachnoid hemorrhage in the left frontal lobe .
tar:  Head computed tomography showed a small contusion with a small subarachnoid bleed in the left frontal lobe .
out:  head  body imaging showed a small contusion with a small subarachnoid  bleeding in the left frontal lobe .

src:  Of note , patient recently had admission Date range 46889 due to abscess following cat scratch and discharged on augmentin and doxycycline for 10 days total ( history 10 days total ) .
tar:  Of note , patient recently had admission Date range 46889 due to abscess following bacterial infection spread by cats and discharged on augmentin and doxycycline for 10 days total ( history 10 days total ) .
out:  of note , patient recently had admission 

In [11]:
for len_current in len_spilt:
    index = np.where((len_sen>=len_current[0])&(len_sen<len_current[1]))
    ref = test_targets[index].tolist()
    hyp = test_subjects[index].tolist()
    open('tmp/hyp.txt', 'w').writelines([x for x in hyp])
    ref0 = [x[0] for x in ref]
    ref1 = [x[1] for x in ref]
    ref2 = [x[2] for x in ref]
    ref3 = [x[3] for x in ref]
    open('tmp/ref0.txt', 'w').writelines([x + '\n' for x in ref0])
    open('tmp/ref1.txt', 'w').writelines([x + '\n' for x in ref1])
    open('tmp/ref2.txt', 'w').writelines([x + '\n' for x in ref2])
    open('tmp/ref3.txt', 'w').writelines([x + '\n' for x in ref3])

    dics = test_dics[index].tolist()
    
    metrics_dict = compute_metrics(hypothesis='tmp/hyp.txt', references=['tmp/ref0.txt','tmp/ref1.txt','tmp/ref2.txt','tmp/ref3.txt'], no_glove=True, no_overlap=False, no_skipthoughts=True)
    #print(metrics_dict)
    hyp = [nltk.word_tokenize(x) for x in hyp]
    hit = count_hit(hyp, dics)
#     hit=1
    com = count_common(hyp)
    BLEU = (metrics_dict['Bleu_1']+metrics_dict['Bleu_2']+metrics_dict['Bleu_3']+metrics_dict['Bleu_4'])/4
    Ascore = (1+2.25+4)/(4/BLEU+2.25/hit+1/com)
    print("BLEU:%4f, HIT:%4f, COM:%4f, ASCORE:%4f"%(BLEU, hit, com, Ascore))


Bleu_1: 0.809249
Bleu_2: 0.759964
Bleu_3: 0.717941
Bleu_4: 0.679479
METEOR: 0.470794
ROUGE_L: 0.822588
CIDEr: 5.596186
BLEU:0.741658, HIT:0.709753, COM:0.745573, ASCORE:0.731977


In [ ]:
'''
test(2030).txt:
She was intubated and was resuscitated after 10-22 minutes of PEA arrest .
She was intubated and was waken up after 10-22 minutes of PEA[pulseless electrical activity] arrest .
She was intubated and was waken up after 10-22 minutes of unresponsive pulse arrest .

第二步输入：
[
"She was intubated and was resuscitated after 10-22 minutes of PEA arrest",
"PEA: value1; value2; value3; value4",
"She was intubated and was resuscitated after 10-22 minutes of value1 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value2 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value3 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value4 arrest",
]

第二步对四个value的embedding：[[emb1], [emb2], [emb3], [emb4]]
每个embedding在单词的level上取平均：[avg_emb1, avg_emb2, avg_emb3, avg_emb4]
第二步拿到四个value的score：[score1, score2, score3, score4]
score做softmax: sum([soft_score1, ..., soft_score4]) = 1 
叠加：[soft_score1*avg_emb1, ..., soft_score4*avg_emb4]
    = [new_emb1, ..., new_emb4]

第三步输入
She was intubated and was resuscitated after 10-22 minutes of [unused4] arrest
过第三步encoder -->
emb_she emb_was  emb_intubated ............................ emb_unused4 emb_arrest
替换 -->
emb_she emb_was  emb_intubated ............................ sum([new_emb1,...,new_emb4]) emb_arrest






"She was intubated and was resuscitated after 10-22 minutes of value1 arrest" [score1]
"She was intubated and was resuscitated after 10-22 minutes of value2 arrest" [score2]
"She was intubated and was resuscitated after 10-22 minutes of value3 arrest" [score3]
"She was intubated and was resuscitated after 10-22 minutes of value4 arrest" [score4]


第二步输出：She was intubated and was waken up after 10-22 minutes of pulseless electrical activity arrest .
第三步输入：She was intubated and was w



A = [
"She was intubated and was resuscitated after 10-22 minutes of PEA arrest",
"PEA: value1; value2; value3; value4",
"She was intubated and was resuscitated after 10-22 minutes of value1 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value2 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value3 arrest",
"She was intubated and was resuscitated after 10-22 minutes of value4 arrest",
]

EA1 = bert(A)
EA2 = pubmed(A_unused4[0])

dataloader - > 
bert(A), list_new_emb,


src:    Weaned off vent to CPAP and was extubated in the afternoon on 9-2 by the pulmonary team .
tar:    Weaned off vent to continuous positive airway pressure and was extubated in the afternoon on 9-2 by the lung specialist team .
result: weaned off vent to and was extubated in the afternoon on 9 - 2 by the team . 


第二步的输入把src处理成了：
Weaned off vent to [SEP] [MASK] [SEP] and was extubated in the afternoon on 9-2 by the [SEP] [MASK] [SEP] team .

'''